In [3]:
import pandas as pd
child_mortality = pd.read_csv('child-mortality.csv')
child_mortality.head(100)

,Entity,Code,Year,"Child mortality (Select Gapminder, v10) (2017)"
0,Afghanistan,AFG,1957,381.194
1,Afghanistan,AFG,1958,375.190
2,Afghanistan,AFG,1959,369.361
3,Afghanistan,AFG,1960,363.700
4,Afghanistan,AFG,1961,357.500
...,...,...,...,...
95,Albania,ALB,2013,14.900
96,Albania,ALB,2014,14.400
97,Albania,ALB,2015,14.000
98,Albania,ALB,2016,13.500


In [30]:
child_mortality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13512 entries, 0 to 13511
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Entity           13512 non-null  object 
 1   Code             13512 non-null  object 
 2   Year             13512 non-null  int64  
 3   Child_mortality  13512 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 422.4+ KB


### TASK 1 : Minimum values of child mortality in different countries

In [24]:
child_mortality.rename(columns={"Child mortality (Select Gapminder, v10) (2017)": "Child_mortality"}, inplace=True)
df = child_mortality[["Entity", "Code", "Child_mortality"]]
df.groupby("Code").min().sort_values("Child_mortality")

,Entity,Child_mortality
Code,,
ISL,Iceland,2.100
HKG,Hong Kong,2.282
FIN,Finland,2.300
SVN,Slovenia,2.300
LUX,Luxembourg,2.400
...,...,...
MLI,Mali,110.600
SLE,Sierra Leone,113.500
CAF,Central African Republic,123.600


### TASK 2 : Least value of CM in countries in 2016

In [34]:
# Check if 2017 is available for all countries
df = child_mortality[child_mortality["Year"] == 2016]
df = df[["Entity", "Code", "Child_mortality"]]  # Use only relevant columns
df.sort_values("Child_mortality", inplace=True)

df.head()

,Entity,Code,Child_mortality
5528,Iceland,ISL,2.100
5217,Hong Kong,HKG,2.282
10883,Slovenia,SVN,2.300
4087,Finland,FIN,2.300
7229,Luxembourg,LUX,2.400
